In [2]:
import numpy as np
import pandas as pd
from utils import saveObject_ , categoesMapping ,loadObject_


In [2]:
train_df= pd.read_csv('train.csv')

In [58]:
categiesdData=train_df['Category'].apply(categoesMapping)


In [45]:
categiesdData

0         cultural and artistic
1                        sports
2                      economic
3                 miscellaneous
4                        sports
                  ...          
150091            miscellaneous
150092            miscellaneous
150093            miscellaneous
150094    cultural and artistic
150095                political
Name: Category, Length: 150096, dtype: object

In [3]:
from sklearn.preprocessing import LabelEncoder 
# label_encoder object knows  
# how to understand word labels. 
label_encoder = LabelEncoder() 
  
# Encode labels in column 'species'. 
# E:\term7\ML\FinalProject\classifierServer\rawcategories.csv
rawCats = pd.read_csv('./rawcategories.csv',header=None)
categiesdData = rawCats[0].apply(categoesMapping)
categiesdData= label_encoder.fit_transform(categiesdData) 


In [12]:
saveObject_(label_encoder,'labelEncoder')


In [51]:
import re

def remove_urls(text):
        url_pattern = re.compile(r'https?://\S+')
        return url_pattern.sub('', text)
    
train_df['Text'] = train_df['Text'].apply(remove_urls)

train_df['Text'] = train_df['Text'].replace(to_replace=r'[^\w\s]', value='', regex=True)

train_df['Text'] = train_df['Text'].replace(to_replace=r'\d', value='', regex=True)

train_df['Text'] = train_df['Text'].replace(to_replace=r'\n', value='', regex=True)

processedData=train_df['Text']


In [6]:

stopWordsFile2 = open('./persian-stopwords/persian', encoding='utf-8')
combined = stopWordsFile2.read().split('\n')
stopWords = set(combined)

In [58]:

processedData=processedData.apply(lambda x: [word for word in x if word not in stopWords])



In [59]:
pd.DataFrame(processedData).to_csv('finalTexts.csv',header=None,index=None)


In [4]:
df = pd.read_csv('finalTexts.csv',header=None)

In [5]:
df[0].head()

0    ['خبرنامه', 'دانشگاه', 'علم', 'صنعت', 'ايران',...
1    ['پايان', 'سال', 'دهها', 'زمين', 'فوتبال', 'سا...
2    ['انجمن', 'توليدكنندگان', 'تجهيزات', 'صنعت', '...
3    ['كرتين', 'براي', 'سومين', 'وزير', 'كانادا', '...
4    ['رفقا', 'نمايندگان', 'اروپاي', 'شرقي', 'جام',...
Name: 0, dtype: object

In [6]:
texts=df[0].apply(lambda text : eval(text))


In [8]:
saveObject_(texts,'textsFile')


In [4]:
texts=loadObject_('textsFile')

In [7]:
texts=texts.apply(lambda x: [word for word in x if word not in stopWords])

In [8]:
texts.head()


0    [خبرنامه, دانشگاه, علم, صنعت, ايران, شماره, يا...
1    [پايان, سال, دهها, زمين, فوتبال, سالن, ورزش, ك...
2    [انجمن, توليدكنندگان, تجهيزات, صنعت, نفت, تشكي...
3    [كرتين, سومين, وزير, كانادا, ژان, كرتين, وزير,...
4    [رفقا, نمايندگان, اروپاي, شرقي, جام, بابك, كما...
Name: 0, dtype: object

In [9]:
from sklearn.model_selection import train_test_split
X_train , x_test ,y_train ,y_test = train_test_split(texts,categiesdData,test_size=0.2,random_state=42)


In [10]:
from utils import VECTOR_SIZE, WINDOW, SEED, MIN_COUNT, WORKERS

In [12]:
from gensim.models import Word2Vec
from utils import W2VMODEL
w2v_model = Word2Vec(X_train, vector_size=VECTOR_SIZE, window=WINDOW,seed=SEED, min_count=MIN_COUNT, workers=WORKERS)


c:\Users\Eniac\anaconda3\lib\site-packages\requests\__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [13]:
saveObject_(w2v_model,W2VMODEL)

In [14]:
def vectorize(sentence):
    words_vecs = [w2v_model.wv[word] for word in sentence if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(VECTOR_SIZE)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)



X_train = np.array([vectorize(sentence) for sentence in X_train])
x_test = np.array([vectorize(sentence) for sentence in x_test])


In [15]:
from sklearn.linear_model import LogisticRegression
from utils import LOGISTIC_REGRESSION_MODEL

clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(x_test)


c:\Users\Eniac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
saveObject_(clf,LOGISTIC_REGRESSION_MODEL)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred,average='micro'))
print('Recall:', recall_score(y_test, y_pred,average='micro'))
print('F1 score:', f1_score(y_test, y_pred,average='weighted'))

Accuracy: 0.6869753497668221
Precision: 0.6869753497668221
Recall: 0.6869753497668221
F1 score: 0.7042057688285234


In [18]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=15,min_samples_leaf=5,class_weight='balanced')
rfc.fit(X_train, y_train)
rfc_y_pred = clf.predict(x_test)


In [ ]:
saveObject_(rfc,'rfclassifier')

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy:', accuracy_score(y_test, rfc_y_pred))
print('Precision:', precision_score(y_test, rfc_y_pred,average='weighted'))
print('Recall:', recall_score(y_test, rfc_y_pred,average='weighted'))
print('F1 score:', f1_score(y_test, rfc_y_pred,average='weighted'))

Accuracy: 0.6869753497668221
Precision: 0.7840295195880695
Recall: 0.6869753497668221
F1 score: 0.7042057688285234


In [12]:
import re
newText= '''برنامه درسی دو ساله زبان اشاره بریتانیایی شامل دست‌کم ۷۵۰ نشانه است و مخصوص دانش‌آموزانی است که هیچ دانش قبلی از این زبان ندارند. در پایان دوره، دانش‌آموزان باید بتوانند به‌طور موثر با زبان اشاره بریتانیایی ارتباط برقرار کنند.

در این دوره دانش‌آموزان همچنین تاریخچه زبان اشاره بریتانیایی، اهمیت زبان‌های اشاره و چگونگی شکل‌گیری این زبان را یاد می‌گیرند.

گنجاندن زبان اشاره بریتانیایی در برنامه درسی، یک پیروزی برای افرادی است که سعی داشتند استفاده از این زبان را فراتر از جامعه ناشنوایان گسترش دهند.'''
text = re.sub(r'[^\w\s]', '', newText)
text = re.sub(r'\d', '', newText)

textList=[word for word in text.split(' ') if word]
stopWordsFile = open('./stopwords.dat', encoding='utf-8')
stopWordsFile2 = open('./persian-stopwords/persian', encoding='utf-8')
combined = stopWordsFile.read().split('\n') + stopWordsFile2.read().split('\n')
stopWords = set(combined)

cleanList = [word for word in textList if word not in stopWords]

x_test = np.array([vectorize(cleanList)])
print(
    # label_encoder.inverse_transform(rfc.predict(x_test)),
    label_encoder.inverse_transform(clf.predict(x_test)))

['social']


In [17]:
label_encoder.classes_

array(['cultural and artistic', 'economic', 'environment',
       'miscellaneous', 'political', 'social', 'sports', 'technological',
       'tourism'], dtype=object)

In [62]:
from utils import LOGISTIC_REGRESSION_MODEL , saveObject
saveObject(clf,LOGISTIC_REGRESSION_MODEL)